# Municipalities
This notebook has the purpose to map all municipalities to their coordinates, as the weather will be queried by the coordinates.

In [ ]:
import json
import requests as rq
import pandas as pd

In [ ]:
credentials = {}
try:
    with open("../Ressources/credentials.json") as file:
        credentials = json.load(file)
except FileNotFoundError:
    print("Credentials file not found")

In [ ]:
# Sample Request with API Ninjas geocoding
municipality = 'Kappel am Albis'
api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
response = rq.get(api_url + municipality, headers={'X-Api-Key': credentials['ApiNinjasKey']})
if response.status_code == rq.codes.ok:
    print(response)
else:
    print("Error:", response.status_code, response.text)


In [ ]:
response.json()[0]

In [ ]:
## Load municipalities
municipalities = pd.read_excel('../data_engineering/Data/municipalities/be-b-00.04-agv-20050313.xlsx', sheet_name='Gemeindeliste-Liste d. communes')

In [ ]:
new_mun_coordinates = municipalities['GDENAME'].tolist()
problems = []
len_todo = len(new_mun_coordinates)
for i, municipality in enumerate(new_mun_coordinates):
    api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
    response = rq.get(api_url + municipality, headers={'X-Api-Key': credentials['ApiNinjasKey']})
    if response.status_code == rq.codes.ok:
        if len(response.json())== 0:
            problems.append(municipality)
            print("Error with ", municipality)
            continue
        name = response.json()[0].get("name")
        latitude = response.json()[0].get("latitude")
        longitude = response.json()[0].get("longitude")
        new_mun_coordinates.append([name, latitude, longitude])
    else:
        print("Error:", response.status_code, response.text)
        problems.append(municipality)
    if i %50== 0:
        print(i, " of ", len_todo, " done")

columns = ['Municipality', 'Latitude', 'Longitude']
df_mun_coordinates = pd.DataFrame.from_records(new_mun_coordinates, columns=columns)
df_mun_coordinates.to_csv('../data_engineering/Data/municipality_to_latitude_longitude.csv', index=False)
pd.DataFrame(problems).to_csv('../data_engineering/Data/municipalities_coordinates_problems.csv', index=False)

## Validation
- added the "problematic" municipalities manually
- NO municipality twice in the list, otherwise delete
- join with [municipalities](data_engineering/Data/municipalities/be-b-00.04-agv-20050313.xlsx) as this is the main file given by the swiss government.

In [ ]:
df_final_mun = pd.read_csv('../data_engineering/Data/municipalities/municipality_to_latitude_longitude.csv')
df_main_mun = pd.read_excel('../data_engineering/Data/municipalities/be-b-00.04-agv-20050313.xlsx', sheet_name='Gemeindeliste-Liste d. communes')

In [ ]:
df_final_mun[df_final_mun.duplicated(['Municipality'])]

In [ ]:
df_main_mun = df_main_mun['GDENAME']

In [ ]:
df_main_mun = df_main_mun.to_frame()

In [ ]:
df_merged = df_main_mun.merge(df_final_mun, left_on='GDENAME', right_on='Municipality', how='left')

In [ ]:
df_merged = df_merged[['Municipality', 'Latitude', 'Longitude']]
df_merged.to_csv('../data_engineering/Data/municipalities/municipality_coordinates.csv')

In [ ]:
nan_rows = df_merged[df_merged['Latitude'].isna() | df_merged['Longitude'].isna()]
nan_rows

Suraua was updated directly in the data_engineering/Data/municipalities/municipality_coordinates.csv file!